In [2]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import TickServiceConnection as TSC
import PnLClass
PnL = PnLClass.SparkPnL()


        Brief Introduction:
        PnL per trade: extract_tick_trade, PnL_trade, get_bound_trade and PnL_trade_plot;
        PnL per market: extract_tick_market, PnL_market, get_bound_market and PnL_market_plot.
        


In [2]:
# read filled orders & closing rates
conn = psycopg2.connect(host = 'PDTY3000.ty3.sparksystems.sg',
                                database = 'kgis_db', 
                                user = 'sparksupport',
                                password = 'sparksupport',
                                port = '30101')
        
filled_orders = pd.read_sql_query(
    '''   
    select market, id, side, trade_date, updated_timestamp, order_status, currency_pair, tenor, quantity, limit_price, filled_quantity, filled_price
    from fx.fx_order
    where product_type <> 'FXSWAP' and order_status = 'FILLED' and filled_quantity > 0 and trade_date = '2021-06-22'
    ''',
    con=conn)
# filled_orders['updated_timestamp'] = pd.to_datetime(filled_orders['updated_timestamp'], unit='ms')
# filled_orders = filled_orders.reset_index().drop('index', axis=1)

closing_rates = pd.read_sql_query(
    '''
    select currency_pair, closing_rate 
    from fx.eod_rate 
    where date = '2021-06-21' and tenor = 'SP'
    ''', 
    con=conn)
# date of closing_rates can be decided by dt.datetime.strptime('2021-04-22', "%Y-%m-%d")-dt.timedelta(days = 1)).strftime("%Y-%m-%d")

# close the connetion
conn.close()

In [7]:
def add_position(filled_orders):
    filled_orders['position'] = 1
    cond1 = (filled_orders['market'].str.find('SPARK') == -1) & (filled_orders['side'] == 'BUY')
    cond2 = (filled_orders['market'].str.find('SPARK') != -1) & (filled_orders['side'] == 'SELL')
    filled_orders.loc[cond1 | cond2, 'position'] = -1
    filled_orders['position'] = filled_orders['position'] * filled_orders['filled_quantity']
    return filled_orders

In [4]:
filled_orders_disp = add_position(filled_orders)
filled_orders_disp

,market,id,side,trade_date,updated_timestamp,order_status,currency_pair,tenor,quantity,limit_price,filled_quantity,filled_price,position
0,CS,BGY01083855,SELL,2021-06-22,1624318394156,FILLED,USDJPY,SP,1000.0,110.3580,1000.0,110.3580,1000.0
1,SPARK_PRICING,BGZ070316338,SELL,2021-06-22,1624318394157,FILLED,USDJPY,SP,1000.0,NaN,1000.0,110.3560,-1000.0
2,BAML,BGY04452225,SELL,2021-06-22,1624318394189,FILLED,USDJPY,SP,1000.0,110.3590,1000.0,110.3590,1000.0
3,SPARK_PRICING,BGZ070316336,SELL,2021-06-22,1624318394189,FILLED,USDJPY,SP,1000.0,NaN,1000.0,110.3570,-1000.0
4,JPMC_SG,BGY22273377,SELL,2021-06-22,1624318394225,FILLED,USDJPY,SP,1000.0,110.3600,1000.0,110.3600,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,SPARK_PRICING,BGZ070318734,BUY,2021-06-22,1624395077803,FILLED,USDMXN,SP,1000.0,NaN,1000.0,20.3358,1000.0
2396,UBS,BGY012102819,BUY,2021-06-22,1624395083358,FILLED,USDMXN,SP,1000.0,20.3333,1000.0,20.3333,-1000.0
2397,SPARK_PRICING,BGZ070318738,BUY,2021-06-22,1624395083358,FILLED,USDMXN,SP,1000.0,NaN,1000.0,20.3336,1000.0
2398,BAML,BGY04452431,BUY,2021-06-22,1624395083372,FILLED,USDMXN,SP,1000.0,20.3327,1000.0,20.3327,-1000.0


In [5]:
# check final positions after a day
filled_orders.groupby('currency_pair').agg({'position': 'sum'})

,position
currency_pair,
AUDUSD,0.0
EURGBP,0.0
EURJPY,0.0
EURUSD,0.0
EURZAR,0.0
GBPJPY,0.0
GBPUSD,0.0
USDCAD,0.0
USDCHF,0.0


In [6]:
# currently hedge without limit_pricesetting 
filled_orders.loc[filled_orders['market'] == 'SPARKPRICING', 'limit_price'].unique()

array([], dtype=float64)

In [5]:
def position_weighted_PnL_attribution(filled_orders):
    
    filled_orders = add_position(filled_orders)
    currency_pairs = filled_orders['currency_pair'].unique()
    frames = []
    
    for curr_pair in currency_pairs:
        
        curr_orders = filled_orders.loc[filled_orders['currency_pair'] == curr_pair,].sort_values(by='updated_timestamp').reset_index().drop('index', axis=1)
        curr_orders['left_position'] = curr_orders['position']
        curr_orders['cumulative_position'] = curr_orders['position'].cumsum()
        curr_orders['PnL'] = 0
        
        realized_sign = (np.sign(curr_orders['position'].shift(-1)) * np.sign(curr_orders['cumulative_position'])).shift()
        realized_indexes = np.insert(np.where(realized_sign == -1), 0, -1)
        
        open_indexes = []

        for i in range(1, len(realized_indexes)): 

            open_indexes.extend(range(realized_indexes[i-1] + 1, realized_indexes[i]))
            open_orders = curr_orders.iloc[open_indexes,].reset_index().drop('index', axis=1)
            open_orders['position'] = open_orders['left_position']
            open_orders['cumulative_position'] = open_orders['position'].cumsum()
            last_cum_pos = open_orders.loc[len(open_orders['cumulative_position'])-1, 'cumulative_position']

            ri = realized_indexes[i]
            rpr = curr_orders.loc[ri, 'filled_price']
            rpos = curr_orders.loc[ri, 'position']

            weights_indexes = open_indexes
            weights_indexes.append(ri)

            if abs(last_cum_pos) < abs(rpos):
                open_orders['left_position'] = 0
                curr_orders.loc[open_indexes, 'left_position'] = 0
                curr_orders.loc[ri, 'left_position'] = np.sign(rpos) * (abs(rpos) - abs(last_cum_pos))
                open_indexes = [ri]
            elif abs(last_cum_pos) == abs(rpos):
                open_orders['left_position'] = 0
                curr_orders.loc[open_indexes, 'left_position'] = 0
                curr_orders.loc[ri, 'left_position'] = 0
                open_indexes = []
            else:
                filled_index = sum(abs(open_orders['cumulative_position']) < abs(rpos))
                filled_index_position = curr_orders.loc[open_indexes[filled_index], 'cumulative_position']
                open_orders.loc[:filled_index, 'left_position'] = 0
                open_orders.loc[filled_index, 'left_position'] = np.sign(filled_index_position) * (abs(filled_index_position) - abs(rpos))
                curr_orders.loc[open_indexes[:filled_index], 'left_position'] = 0
                curr_orders.loc[open_indexes[filled_index], 'left_position'] = np.sign(filled_index_position) * (abs(filled_index_position) - abs(rpos))
                curr_orders.loc[ri, 'left_position'] = 0
                open_indexes = open_indexes[filled_index:]

            rpnl = sum((open_orders['left_position'] - open_orders['position']) * open_orders['filled_price']) + (curr_orders['left_position'][ri] - rpos) * rpr
            weights_orders = open_orders
            weights_orders.loc[len(weights_orders.index)] = list(curr_orders.iloc[ri,])
            weights = abs(weights_orders['position'] - weights_orders['left_position']) / sum(abs(weights_orders['position'] - weights_orders['left_position'])) # can be modified using self-defined functions
            
            for i in range(len(weights_indexes)): 
                curr_orders.loc[weights_indexes[i], 'PnL'] += weights[i] * rpnl
        
        # combine all the currency orders together
        frames.append(curr_orders)
    
    return_orders = pd.concat(frames)
    return return_orders

In [8]:
%%time
position_weighted_PnL_attribution(filled_orders)

Wall time: 10.2 s


,market,id,side,trade_date,updated_timestamp,order_status,currency_pair,tenor,quantity,limit_price,filled_quantity,filled_price,position,left_position,cumulative_position,PnL
0,CS,BGY01083855,SELL,2021-06-22,1624318394156,FILLED,USDJPY,SP,1000.0,110.35800,1000.0,110.35800,1000.0,0.0,1000.0,-1.00
1,SPARK_PRICING,BGZ070316338,SELL,2021-06-22,1624318394157,FILLED,USDJPY,SP,1000.0,NaN,1000.0,110.35600,-1000.0,0.0,0.0,-1.00
2,BAML,BGY04452225,SELL,2021-06-22,1624318394189,FILLED,USDJPY,SP,1000.0,110.35900,1000.0,110.35900,1000.0,0.0,1000.0,-1.00
3,SPARK_PRICING,BGZ070316336,SELL,2021-06-22,1624318394189,FILLED,USDJPY,SP,1000.0,NaN,1000.0,110.35700,-1000.0,0.0,0.0,-1.00
4,JPMC_SG,BGY22273377,SELL,2021-06-22,1624318394225,FILLED,USDJPY,SP,1000.0,110.36000,1000.0,110.36000,1000.0,0.0,1000.0,-1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,SPARK_PRICING,BGZ070318634,SELL,2021-06-22,1624388875610,FILLED,EURUSD,SP,1000.0,NaN,1000.0,1.19500,-1000.0,0.0,0.0,-0.01
88,JPMC_SG,BGY22274451,SELL,2021-06-22,1624388964588,FILLED,EURUSD,SP,1000.0,1.19514,1000.0,1.19514,1000.0,0.0,1000.0,-0.02
89,SPARK_PRICING,BGZ070318668,SELL,2021-06-22,1624388964589,FILLED,EURUSD,SP,1000.0,NaN,1000.0,1.19510,-1000.0,0.0,0.0,-0.02
90,SPARK_PRICING,BGZ070318670,SELL,2021-06-22,1624388964641,FILLED,EURUSD,SP,1000.0,NaN,1000.0,1.19512,-1000.0,0.0,-1000.0,-0.01


In [11]:
test = pd.DataFrame({'market': ['CS', 'JPMC', 'SPARK_PRICING', 'SPARK_PRICING'], 'side': ['SELL', 'SELL', 'SELL', 'SELL'], 'updated_timestamp': [1,2,3,4], 'currency_pair': ['USDJPY', 'USDJPY','USDJPY','USDJPY'],
                     'tenor': ['SP', 'SP', 'SP', 'SP'], 'filled_quantity': [1,4,2,3], 'filled_price': [110, 111, 108, 109]})
test

,market,side,updated_timestamp,currency_pair,tenor,filled_quantity,filled_price
0,CS,SELL,1,USDJPY,SP,1,110
1,JPMC,SELL,2,USDJPY,SP,4,111
2,SPARK_PRICING,SELL,3,USDJPY,SP,2,108
3,SPARK_PRICING,SELL,4,USDJPY,SP,3,109


In [12]:
position_weighted_PnL_attribution(test)

,market,side,updated_timestamp,currency_pair,tenor,filled_quantity,filled_price,position,left_position,cumulative_position,PnL
0,CS,SELL,1,USDJPY,SP,1,110,1,0,1,-1.25
1,JPMC,SELL,2,USDJPY,SP,4,111,4,0,5,-4.25
2,SPARK_PRICING,SELL,3,USDJPY,SP,2,108,-2,0,3,-2.50
3,SPARK_PRICING,SELL,4,USDJPY,SP,3,109,-3,0,0,-3.00


In [9]:
def position_TCA_weighted_PnL_attribution(filled_orders, tickdb, weighted_function, closing_rates, T):
    # parameter of weighted_function are filled_positions, TCA_effect, realized_pnl
    
    filled_orders = add_position(filled_orders)
    currency_pairs = filled_orders['currency_pair'].unique()
    frames = []
    
    for curr_pair in currency_pairs:
        
        curr_orders = filled_orders.loc[filled_orders['currency_pair'] == curr_pair,].sort_values(by='updated_timestamp').reset_index().drop('index', axis=1)
        curr_orders['left_position'] = curr_orders['position']
        curr_orders['cumulative_position'] = curr_orders['position'].cumsum()
        curr_orders['PnL'] = 0
        curr_orders['TCA'] = 0
        for i in range(curr_orders.shape[0]):
            curr_orders.loc[i, 'TCA'] = PnL.PnL_trade(tickdb, curr_orders, closing_rates, curr_orders.loc[i, 'id'], [T])[0]
        
        realized_sign = (np.sign(curr_orders['position'].shift(-1)) * np.sign(curr_orders['cumulative_position'])).shift()
        realized_indexes = np.insert(np.where(realized_sign == -1), 0, -1)
        
        open_indexes = []

        for i in range(1, len(realized_indexes)): 

            open_indexes.extend(range(realized_indexes[i-1] + 1, realized_indexes[i]))
            open_orders = curr_orders.iloc[open_indexes,].reset_index().drop('index', axis=1)
            open_orders['position'] = open_orders['left_position']
            open_orders['cumulative_position'] = open_orders['position'].cumsum()
            last_cum_pos = open_orders.loc[len(open_orders['cumulative_position'])-1, 'cumulative_position']

            ri = realized_indexes[i]
            rpr = curr_orders.loc[ri, 'filled_price']
            rpos = curr_orders.loc[ri, 'position']

            weights_indexes = open_indexes
            weights_indexes.append(ri)

            if abs(last_cum_pos) < abs(rpos):
                open_orders['left_position'] = 0
                curr_orders.loc[open_indexes, 'left_position'] = 0
                curr_orders.loc[ri, 'left_position'] = np.sign(rpos) * (abs(rpos) - abs(last_cum_pos))
                open_indexes = [ri]
            elif abs(last_cum_pos) == abs(rpos):
                open_orders['left_position'] = 0
                curr_orders.loc[open_indexes, 'left_position'] = 0
                curr_orders.loc[ri, 'left_position'] = 0
                open_indexes = []
            else:
                filled_index = sum(abs(open_orders['cumulative_position']) < abs(rpos))
                filled_index_position = curr_orders.loc[open_indexes[filled_index], 'cumulative_position']
                open_orders.loc[:filled_index, 'left_position'] = 0
                open_orders.loc[filled_index, 'left_position'] = np.sign(filled_index_position) * (abs(filled_index_position) - abs(rpos))
                curr_orders.loc[open_indexes[:filled_index], 'left_position'] = 0
                curr_orders.loc[open_indexes[filled_index], 'left_position'] = np.sign(filled_index_position) * (abs(filled_index_position) - abs(rpos))
                curr_orders.loc[ri, 'left_position'] = 0
                open_indexes = open_indexes[filled_index:]

            rpnl = sum((open_orders['left_position'] - open_orders['position']) * open_orders['filled_price']) + (curr_orders['left_position'][ri] - rpos) * rpr
            weights_orders = open_orders
            weights_orders.loc[len(weights_orders.index)] = list(curr_orders.iloc[ri,])
            # can be modified using self-defined functions
            weighted_pnl = weighted_function(abs(weights_orders['position'] - weights_orders['left_position']), weights_orders['TCA'], rpnl)
            
            for i in range(len(weights_indexes)): 
                curr_orders.loc[weights_indexes[i], 'PnL'] += weighted_pnl[i]
        
        # combine all the currency orders together
        frames.append(curr_orders)
    
    return_orders = pd.concat(frames)
    return return_orders

In [10]:
def weighted_function(filled_positions, TCA_effect, realized_pnl): # to be modified
    # all the orders are attributed same sign as realized_ give more weights to positive TCA_effect
    
    weighted_pnl = filled_positions
    weighted_impact = filled_positions * TCA_effect
    neg_TCA_indexes = TCA_effect[TCA_effect<0].index.values
    nonneg_TCA_indexes = TCA_effect[TCA_effect>=0].index.values
    
    neg_weights = weighted_impact / sum(abs(weighted_impact)) 
    neg_weights = abs(neg_weights[neg_TCA_indexes]).reset_index(drop=True) * 0.2
    pos_weights = weighted_impact[nonneg_TCA_indexes].reset_index(drop=True)
    pos_weights = (1 - sum(neg_weights)) * pos_weights / sum(abs(pos_weights))
    
    for i in range(len(nonneg_TCA_indexes)):
        weighted_pnl[nonneg_TCA_indexes[i]] = pos_weights[i] * realized_pnl
    for i in range(len(neg_TCA_indexes)):
        weighted_pnl[neg_TCA_indexes[i]] = neg_weights[i] * realized_pnl

    if realized_pnl < 0:
        weighted_pnl = - weighted_pnl

    return weighted_pnl

In [11]:
db_path = 'C:/Users/yuxin.mei-t/Desktop/tickAnalysis/PnL/SparkTick'
java_home_path = 'C:/Program Files/Java/jdk-11'
jar_path = 'C:/Users/yuxin.mei-t/Desktop/tickAnalysis/PnL/sparktick-4.7.0-SNAPSHOT.jar'

TSC.startLocalSession(db_path, java_home_path, jar_path)

Connected to JVM


In [12]:
%%time
EURUSD_db = TSC.queryTickData('FULL_AMOUNT', 'EURUSD.SP', 1, '2021-06-21 13:00:00', '2021-06-23 13:00:00')
EURUSD = PnL.extract_tick_trade(EURUSD_db)

Wall time: 1min 22s


In [13]:
%%time
# an example considering EURUSD trades
weights_orders = filled_orders.loc[filled_orders['currency_pair'] == 'EURUSD',].reset_index().drop('index', axis=1)
position_TCA_weighted_PnL_attribution(weights_orders, EURUSD, weighted_function, closing_rates, 1000)

Wall time: 6.75 s


,market,id,side,trade_date,updated_timestamp,order_status,currency_pair,tenor,quantity,limit_price,filled_quantity,filled_price,position,left_position,cumulative_position,PnL,TCA
0,JPMC_SG,BGY22273551,BUY,2021-06-22,1624354518071,FILLED,EURUSD,SP,1000.0,1.18910,1000.0,1.18910,-1000.0,0.0,-1000.0,0.017000,5.0
1,SPARK_PRICING,BGZ070316958,BUY,2021-06-22,1624354518071,FILLED,EURUSD,SP,1000.0,NaN,1000.0,1.18912,1000.0,0.0,0.0,0.003000,-15.0
2,UBS,BGY012102591,BUY,2021-06-22,1624354531534,FILLED,EURUSD,SP,1000.0,1.18867,1000.0,1.18867,-1000.0,0.0,-1000.0,0.022857,60.0
3,SPARK_PRICING,BGZ070316960,BUY,2021-06-22,1624354531559,FILLED,EURUSD,SP,1000.0,NaN,1000.0,1.18871,1000.0,0.0,0.0,0.017143,45.0
4,JPMC_SG,BGY22273553,BUY,2021-06-22,1624354531640,FILLED,EURUSD,SP,1000.0,1.18867,1000.0,1.18867,-1000.0,0.0,-1000.0,0.027273,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,SPARK_PRICING,BGZ070318634,SELL,2021-06-22,1624388875610,FILLED,EURUSD,SP,1000.0,NaN,1000.0,1.19500,-1000.0,0.0,0.0,0.001000,-5.0
88,JPMC_SG,BGY22274451,SELL,2021-06-22,1624388964588,FILLED,EURUSD,SP,1000.0,1.19514,1000.0,1.19514,1000.0,0.0,1000.0,0.033000,5.0
89,SPARK_PRICING,BGZ070318668,SELL,2021-06-22,1624388964589,FILLED,EURUSD,SP,1000.0,NaN,1000.0,1.19510,-1000.0,0.0,0.0,0.007000,-35.0
90,SPARK_PRICING,BGZ070318670,SELL,2021-06-22,1624388964641,FILLED,EURUSD,SP,1000.0,NaN,1000.0,1.19512,-1000.0,0.0,-1000.0,0.003000,-15.0
